In [ ]:
from bs4 import BeautifulSoup
from urllib.request import urlopen
import urllib 
from urllib.request import Request
from user_agent import generate_user_agent
import re
from urllib.parse import urljoin
import pandas as pd
import folium

In [ ]:
url = "https://www.chicagomag.com/chicago-magazine/november-2012/best-sandwiches-chicago/"
req = Request(url=url, headers={'User-Agent':generate_user_agent()})
page =urlopen(req)
soup = BeautifulSoup(page, "html.parser")
print(soup.prettify())

In [ ]:
prefix = "https://www.chicagomag.com/"
item = soup.find_all('div', 'sammy')[0]
for item in soup.find_all('div','sammy'):
    print(urljoin(prefix, item.find('a')['href']))

In [ ]:
rank_list = []
menu_list = []
cafe_list = []
link_list = []

for item in soup.find_all('div','sammy'):
    rank_list.append(item.find(class_='sammyRank').string)
    text_list = re.split(('\n'), item.find(class_="sammyListing").get_text())
    menu_list.append(text_list[0])
    cafe_list.append(text_list[1])
    link_list.append(urljoin(prefix, item.find('a')['href']))

data = {'Rank':rank_list, 'Cafe':cafe_list, 'Menu':menu_list,'URL':link_list}
df = pd.DataFrame(data)
df.to_csv("../data/chicago_sanwichs.csv", sep=",", encoding="utf-8")
df.head()

In [ ]:
from tqdm import tqdm

price_list = []
address_list = []

for idx, row in tqdm(df[:50].iterrows(), total=len(df.index)):
    req = Request(url=row['URL'], headers={'User-Agent':generate_user_agent()})
    page =urlopen(req)
    soup = BeautifulSoup(page, "html.parser")

    text = re.split('.,', soup.find('p','addy').get_text())[0]
    price = re.search('\$\d+\.(\d+)?', text).group()
    address = text[len(price) + 1:]
    
    price_list.append(price)
    address_list.append(address)

df['Price'] = price_list
df['Address'] = address_list
df.set_index('Rank', inplace=True)
df.head()



In [ ]:
df = df.loc[:, ['Cafe', 'Menu', 'Price', 'Address']]
df.to_csv("../data/chicago_sandwiches_50.csv", sep=",", encoding="utf-8")
df.head()